## Recommender System - Marketplace Matching

In this notebook, we will: 
- Clean textual data from user-input verbatim posts
- Use a Word2Vec model to calculate document similarities 
- Sort the most similar user input to our training data in order to recommend similar products 
- Save this model in a format that allows us to refresh the testing data

The goal of this project is to create a recommender system to help Pangeans find the right "project" for them, given their profile information. Here, we are using legacy data that is from Pangea V2, when Pangeans were allowed to post services and requests, as well as purchase items on the platform. We are using the User-Inputted Titles to suggest similar services, or in V3, similar "projects". 

In [1]:
#Importing Libraries
import numpy as np
import pandas as pd

import sys
from pandas import DataFrame

import json
from pandas.io.json import json_normalize
import csv

import matplotlib.pyplot as plt
%matplotlib inline 
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

from gensim import corpora
from collections import defaultdict
from pprint import pprint

from gensim.models.word2vec import Word2Vec
from gensim.test.utils import common_texts, get_tmpfile

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import common_texts

import re
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
nltk.download('stopwords') ###

import gensim 
from gensim.models import KeyedVectors
import gensim.downloader as api

from operator import itemgetter, attrgetter

from gensim.models.doc2vec import Doc2Vec

import os, sys

from operator import add

from sklearn.preprocessing import normalize

import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from adjustText import adjust_text

from sklearn.manifold import TSNE

import pickle

import json
from pprint import pprint
from pandas.io.json import json_normalize

import operator

import csv

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/angelateng/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True)

In [31]:
def vectorize_and_store_existing_titles():
    
    raw = pd.read_csv("allPostData.csv", header=0);
    
    #we can replace this with a filepath in the future
    titles = raw['title'];
    post_titles = [title for title in titles];
    post_titles = set(post_titles);
    
    tokens = [[word for word in title.lower().split()] for title in post_titles];
    
    clean_words = [[word for word in title if word.isalpha()] for title in tokens];
    stoplist = set(stopwords.words('english'));
    
    titles_nostopwords = [[word for word in title if word not in stoplist] for title in clean_words];   
    #print(len(titles_nostopwords))
    
    filtered_word_list = [[word for word in title if word in model.vocab] for title in titles_nostopwords];
    #print(len(filtered_word_list))
    
    dictionary = dict(zip(post_titles, filtered_word_list))
    
    #title_vectors = {}
    
    #print(len(dictionary.keys()))
    
    #print(len(set(titles)))
    #print(len(set(post_titles)))
    #dupe reqs 
    
    vectorized_titles = pd.DataFrame(columns=["Titles", "Vectors"])
    #print(vectorized_titles)
    
    
    
    for title in post_titles: 
        word_vecs = [model[word] for word in dictionary[title]]
        #print(len(word_vecs))
        if len(word_vecs) == 0:
            title_vec = [np.zeros(300)]
        else: 
            title_vec = normalize(sum(word_vecs).reshape(1, -1))
            #print(title_vec)
       
        vectorized_titles = vectorized_titles.append({'Titles': title, 'Vectors': title_vec}, ignore_index=True)
       
    
    
    #dictionary = dict(zip(post_titles, filtered_word_list))

    #can also replace filepath in the future
    #titles.to_pickle("/Users/angelateng/Dropbox/AMBER/SharpestMinds/raw_titles.pkl")
    vectorized_titles.to_pickle("/Users/angelateng/Dropbox/AMBER/SharpestMinds/vectorized_titles.pkl")
    #print(vectorized_titles)
    #print(title_vec)
    #print(word_vecs)
    return(vectorized_titles)

#vectorize_and_store_existing_titles()


In [32]:
#pd.read_pickle("/Users/angelateng/Dropbox/AMBER/SharpestMinds/vectorized_titles.pkl")
#sanity check
vectorize_and_store_existing_titles()

,Titles,Vectors
0,Bartender,"[[-0.0146799665, -0.09691628, -0.016318992, 0...."
1,Birthday Planner Wanted,"[[0.03388815, -0.011697535, 0.037889402, 0.047..."
2,Investment Advice,"[[-0.026787853, 0.004830026, 0.007599983, -0.0..."
3,Idea Help,"[[0.03598612, 0.023872536, 0.0220247, 0.074697..."
4,Looking For Graphic Design,"[[-0.010068101, 0.039581455, 0.026983991, 0.02..."
5,Real estate photography,"[[0.08817142, 0.042848743, -0.06355511, 0.0254..."
6,SolidWorks help,"[[0.011597187, 0.06816306, -0.061062742, 0.136..."
7,Friends In Ohio,"[[0.033710547, -0.014110273, 0.005923425, 0.15..."
8,Dulcimer player,"[[0.07746159, -0.07911178, 0.0013589753, 0.039..."
9,Diamond snapback,"[[-0.06737626, -0.004137139, -0.041272886, 0.0..."


In [33]:
#can change this directory later


#test post 
with open('firstPost.json') as fresh_data:
    user_post = json.load(fresh_data)

#user_post = json.loads('firstPost.json')
#print(user_post)

#should only take in a single post at a time 

def vectorize_new_title(user_post):
    json_df = pd.DataFrame.from_dict(json_normalize(user_post), orient='columns')
    #print(json_df)
    
    title = json_df["title"]
    #print("this is the title", title)
    
    #json_post_titles = [title in title];
    json_post_titles = [title for title in title];
    json_tokens = [[word for word in title.lower().split()] for title in json_post_titles];
    #json_post_titles = set(json_post_titles)
    #print(json_post_titles)
    
    
    
    #json_clean_words = [[word for word in title if word.isalpha()] for title in json_post_titles];
    json_clean_words = [[word for word in title if word.isalpha()] for title in json_tokens];
    #print(json_clean_words)
    
    stoplist = set(stopwords.words('english'));
    json_titles_nostopwords = [[word for word in title if word not in stoplist] for title in json_clean_words];   
    
    
    json_filtered_word_list = [[word for word in title if word in model.vocab] for title in json_titles_nostopwords];
    #print(json_filtered_word_list)
    
    json_title_vectors = {}
    
    json_vectorized_titles = pd.DataFrame(columns=["Titles", "Vectors"])
    #print(json_vectorized_titles)
    
    
    for title in json_filtered_word_list: 
        json_word_vecs = [model[word] for word in title]
        if len(json_word_vecs) == 0:
            json_title_vec = [np.zeros(300)]
        else: 
            json_title_vec = normalize(sum(json_word_vecs).reshape(1, -1))
        
        json_vectorized_titles = json_vectorized_titles.append({'Titles': json_post_titles, 'Vectors': json_title_vec}, ignore_index=True)
    return(json_vectorized_titles)

#vectorize_new_title(json_df)
#why is it a dictionary in a dictionary?? double brackets

In [34]:
vectorize_new_title(user_post)
#testing on just 1 entry 

,Titles,Vectors
0,[Teach Me How To Cook!],"[[0.042356346, -0.013683017, 0.07774045, 0.028..."


In [78]:
vectorized_title = vectorize_new_title(user_post)
#print(vectorized_title)

def rank_existing_titles(vectorized_title):
    #loop over all keys in dict 
    ranked_titles = {}
    #other_titles = pd.read_pickle("/Users/angelateng/Dropbox/SharpestMinds/raw_titles.pkl")
    other_titles = pd.read_pickle("/Users/angelateng/Dropbox/AMBER/SharpestMinds/vectorized_titles.pkl")
    #pprint(other_titles)
    
    #transpose_other_titles = [[other_titles[j][i] for j in range(len(other_titles))] for i in range(len(other_titles[0]))] 
    #why is other_titles not printing out the vector, and only the titles?? 
    #need to transpose, since current dimensions are (741,2) and (1,2)

    
    #can also use title_vectors.keys() 
    for index,row in other_titles.iterrows():
        print(index)
        print(row)
        print('***')
        
        print(row['Titles'])
        print(row['Vectors'])
        print(vectorized_title)
        print(vectorized_title['Vectors'][0])
        ranked_titles[row['Titles']] = np.dot(row['Vectors'], vectorized_title['Vectors'][0])
        
        
        #ranked_titles[title] = np.dot(transpose_other_titles, vectorized_title)[0]
    #print(other_titles)
        #print(np.dot(other_titles[title], vectorized_title))
    #sorted_title_vecs = sorted(ranked_titles.items(), key=operator.itemgetter(1), reverse=True)
    #pprint(sorted_title_vecs)
    #dicts not ordered 
    #print(other_titles) 
    return(sorted_title_vecs)

#rank_existing_titles(vectorized_title[:10])
#word querying

#should be cleaned titles x cleaned til

In [51]:
other_titles = pd.read_pickle("/Users/angelateng/Dropbox/AMBER/SharpestMinds/vectorized_titles.pkl")
pprint(other_titles[:5])
#print(vectorized_title)

#i think the issue is that the vectors are a dict in dict.... 

                        Titles  \
0                    Bartender   
1      Birthday Planner Wanted   
2           Investment Advice    
3                    Idea Help   
4  Looking For Graphic Design    

                                             Vectors  
0  [[-0.0146799665, -0.09691628, -0.016318992, 0....  
1  [[0.03388815, -0.011697535, 0.037889402, 0.047...  
2  [[-0.026787853, 0.004830026, 0.007599983, -0.0...  
3  [[0.03598612, 0.023872536, 0.0220247, 0.074697...  
4  [[-0.010068101, 0.039581455, 0.026983991, 0.02...  


In [79]:
rank_existing_titles(vectorized_title)

0
Titles                                             Bartender
Vectors    [[-0.0146799665, -0.09691628, -0.016318992, 0....
Name: 0, dtype: object
***
Bartender
[[-1.46799665e-02 -9.69162807e-02 -1.63189918e-02  3.13552693e-02
  -5.87198660e-02  1.15729444e-01  1.77442320e-02 -3.53459381e-02
   5.10235727e-02  3.91940866e-03  3.79113704e-02 -4.47525196e-02
   7.09769279e-02 -1.29411742e-01  7.35423565e-02  4.50375676e-02
   7.62503128e-03  2.16458249e-03  6.04301542e-02 -1.04327530e-01
   6.12852983e-02  3.24954614e-02  4.81730960e-02 -9.46359038e-02
   3.64861302e-02  1.09030819e-02 -1.19007500e-02  8.95050392e-02
   4.93132845e-02  1.90982092e-02  9.54910461e-03 -4.27571833e-02
   4.95983325e-02  3.13552693e-02 -2.79346928e-02 -6.29955828e-02
   1.23140693e-01  6.64161593e-02  3.00725531e-02  8.38040784e-02
  -4.44674715e-02 -4.93132845e-02  9.23555195e-02  7.66066229e-03
   3.33149731e-03 -9.06452313e-02 -1.68748349e-01 -5.78647219e-02
   4.21870872e-02  3.27805094e-02 -9.57760960e-

ValueError: shapes (1,300) and (1,300) not aligned: 300 (dim 1) != 1 (dim 0)

In [ ]:
other_titles = pd.read_pickle("/Users/angelateng/Dropbox/SharpestMinds/raw_titles.pkl")
other_titles


In [ ]:
vectorized_title = vectorize_new_title(user_post)
print(vectorized_title)

In [ ]:
def generate_recommendations(user_post):
    title = user_post["title"]
    print(title)
    vectorized_title = vectorize_new_title(user_post)
    print(vectorized_title)
    ranked_titles = rank_existing_titles(title, vectorized_title)
    print(ranked_titles)
    
    
    with open("/Users/angelateng/Dropbox/SharpestMinds/ranked_titles.csv", "w", newline='') as myfile:
        wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
        for title in ranked_titles:
            wr.writerow([ranked_titles, title])
    #csv_out = ranked_titles.to_csv('ranked_titles', encoding='utf-8', index=False)
    #writer = csv.writer(ranked_titles, delimiter='', quotechar='"', quoting=csv.QUOTE_ALL)
 
    #for row in reader:
        #writer.writerow(row)

       # with open('csv_out', 'wb') as csv_out:
         #   csv_writer = csv.writer(csv_out, quoting=csv.QUOTE_ALL)
         #   csv_writer.writerow('csv_out')
    return(ranked_titles)
    
    

#vectorize_and_store_existing_titles(); 
#vectorize_new_title(json_df);
#rank_existing_titles(vectorize_new_title);

In [ ]:
#generate_recommendations(json_df)
pprint(generate_recommendations(user_post))

In [ ]:
vectorize_new

In [ ]:
print(other_titles)